![Spark Logo](http://curriculum-release.s3-website-us-west-2.amazonaws.com/wiki-book/general/logo_spark.png)

## Desktop vs Mobile

**Objective:**
Analyze Desktop vs Mobile traffic to English Wikipedia

**Data Source:**
pageviews_by_second ([255 MB](http://datahub.io/en/dataset/english-wikipedia-pageviews-by-second))

**Business Questions:**

* Question # 1) How many total incoming requests were to the mobile site vs the desktop site?
* Question # 2) What is the start and end range of time for the pageviews data? How many days of data is in the DataFrame?
* Question # 3) What is the avg/min/max for the number of requests received for Mobile and Desktop views?
* Question # 4) Which day of the week does Wikipedia get the most traffic?
* Question # 5) Can you visualize both the mobile and desktop site requests in a line chart to compare traffic between both sites by day of the week?
* Question # 6) Why is there so much more traffic on Monday vs. other days of the week?

**Technical Accomplishments:**
- Give a DataFrame a human-readable name when caching
- Cast a String col type into a Timestamp col type
- Browse the Spark SQL API docs
- Learn how to use "Date time functions"
- Create and use a User Defined Function (UDF)
- Join 2 DataFrames
- Visualization

Lets load our file into a SparkRead Dataframe
But what about our schema?  
Notice the "header" option

In [ ]:
val pageviews = sparkSession.read.format("com.databricks.spark.csv")
  .option("header", "true")
  .option("delimiter", "\t")
  .load("data/pageviews-by-second-tsv")

First, lets take a peap at our data

In [ ]:
pageviews.limit(10)

So we got the column names, but it seems everything is defined as string. Lets take a look

In [ ]:
pageviews.printSchema()

Requests as we know (and need) is defentlly not a string so lets define it as a long type and cache our DataFrame into memory

In [ ]:
val pageviewsDF = pageviews.select($"timestamp", $"site", $"requests".cast("long").alias("requests"))

Register the Temporary Table to give the DataFrame a human-readable name in the Storage UI

In [ ]:
pageviewsDF.createOrReplaceTempView("pageviews_by_second")

Lets cache our dataframe and Materialize the cache with a count() action

In [ ]:
pageviewsDF.cache()
pageviewsDF.count()

Verify that the DataFrame is indeed in memory by running a count again

In [ ]:
// This should run in less than a second
pageviewsDF.count()

### How many total incoming requests were to the mobile site vs the desktop site?

First, let's sum up the requests column to see how many total requests are in the dataset

In [ ]:
pageviewsDF.select(sum("requests"))

But how many of the requests were for the mobile site?

***Challenge 1:*** Using just the commands we learned so far, can you figure out how to filter the DataFrame for just mobile traffic and then sum the requests column?

In [ ]:
// Type in your answer here...

In [ ]:
pageviewsDF.filter("site == 'mobile'").select(sum($"requests"))

***Challenge 2***: What about the desktop site? How many requests did it get?

In [ ]:
// Type in your answer here...

In [ ]:
pageviewsDF.filter("site == 'desktop'").select(sum("requests"))

So, about twice as many were for the desktop site

### What is the start and end range of time for the pageviews data? How many days of data is in the DataFrame?

To answer this, we should first convert the timestamp column from a String type to a Timestamp type. Currently the first column of pageviewsDF is typed as a string

In [ ]:
pageviewsDF.printSchema()

Create a new DataFrame, pageviewsOrderedByTimestampDF, that changes the timestamp column from a string data type to a timestamp data type

In [ ]:
val pageviewsTimestampDF = pageviewsDF.select($"timestamp".cast("timestamp").alias("timestamp"), $"site", $"requests")

In [ ]:
pageviewsTimestampDF.printSchema()

How many different years is the data from?

For the next command, we'll use year(), one of the date time function available in Spark. You can review which functions are available for DataFrames in the Spark API doc's SQL functions, under "Date time functions".

In [ ]:
pageviewsTimestampDF.select(year($"timestamp")).distinct()

The data only spans 2015. But which months?

***Challenge 3:*** Can you figure out how to check which months of 2015 the data covers (using the [Spark API docs](https://spark.apache.org/docs/latest/api/python/index.html#org.apache.spark.sql.functions$))?

In [ ]:
// Type in your answer here...

In [ ]:
pageviewsTimestampDF.select(month($"timestamp")).distinct()

The data covers the months you see above.

Similarly, you can discover how many weeks of timestamps are in the data and how many days of data there is:

In [ ]:
// How many weeks of data is there?
pageviewsTimestampDF.select(weekofyear($"timestamp")).distinct()

In [ ]:
// How many days of data is there?
pageviewsTimestampDF.select(dayofyear($"timestamp")).distinct().count()

There is 41 days of data.

### What is the avg/min/max for the number of requests received for Mobile and Desktop views?

To understand our data better, let's look at the average, minimum and maximum number of requests received for mobile, then desktop page views over every 1 second interval:

In [ ]:
// Look at mobile statistics
pageviewsTimestampDF.filter("site = 'mobile'").select(avg("requests"), min("requests"), max("requests"))

In [ ]:
// Look at desktop statistics
pageviewsTimestampDF.filter("site = 'desktop'").select(avg("requests"), min("requests"), max("requests"))

There certainly appears to be more requests for the desktop site.

### Which day of the week does Wikipedia get the most traffic?

Think about how we can accomplish this. We need to pull out the day of the week (like Mon, Tues, etc) from each row, and then sum up all of the requests by day.

First, use the date_format function to extract out the day of the week from the timestamp and rename the column as "Day of week".
Then we'll sum up all of the requests for each day and show the results.

In [ ]:
// Notice the use of alias() to rename the new column
// "E" is a pattern in the SimpleDataFormat class in Java that extracts out the "Day in Week""

// Create a new DataFrame named pageviewsByDayOfWeekDF and cache it
val pageviewsByDayOfWeekDF = pageviewsTimestampDF.groupBy(date_format(($"timestamp"), "E").alias("Day_of_week")).sum()

// Cache the DataFrame with a human-readable name
pageviewsByDayOfWeekDF.createOrReplaceTempView("pageviews_by_DOW")
// spark.cacheTable("pageviews_by_DOW")
pageviewsByDayOfWeekDF.cache()

// Show what is in the new DataFrame
pageviewsByDayOfWeekDF

In [ ]:
// Use orderBy() to sort by day of week
pageviewsByDayOfWeekDF.orderBy("Day_of_week")

Hmm, the ordering of the days of the week is off, because the orderBy() operation is ordering the days of the week alphabetically. Instead of that, let's start with Monday and end with Sunday. To accomplish this, we'll write a short User Defined Function (UDF) to prepend each Day of week with a number.

### User Defined Functions (UDFs)

A UDF lets you code your own logic for processing column values during a DataFrame query.

In [ ]:
def matchDayOfTheWeek(day: String): String = {
    val matchDayOfWeek = Map("Mon"-> "1-Mon", "Tue"-> "2-Tue", "Wed"-> "3-Wed", "Thu"-> "4-Thu", "Fri"->"5-Fri", "Sat"-> "6-Sat", "Sun"-> "7-Sun")
    if (!matchDayOfWeek.contains(day))
        return "UNKNOWN"
    return matchDayOfWeek(day)
}

Test it:

In [ ]:
matchDayOfTheWeek("Tue")

Great, it works! Now define a UDF named prependNumber:

In [ ]:
sparkSession.udf.register("prependNumber", matchDayOfTheWeek(_: String))
val prependNumberUDF = udf(matchDayOfTheWeek(_: String))

Test the UDF to prepend the Day of Week column in the DataFrame with a number:

In [ ]:
pageviewsByDayOfWeekDF.select(prependNumberUDF($"Day_of_week")).limit(10)

Our UDF looks like it's working. Next, let's apply the UDF and also order the x axis from Mon -> Sun:

In [ ]:
pageviewsByDayOfWeekDF.withColumnRenamed("sum(requests)", "total requests")
  .select(prependNumberUDF($"Day_of_week"), $"total requests")
  .orderBy("UDF(Day_of_week)")

Lets do the same thing with SQL. Here our human-readable name comes in handy

In [ ]:
pageviewsByDayOfWeekDF.createOrReplaceTempView("pageviewsByDayOfWeek")
sparkSession.sql("select prependNumber(Day_of_week), * from pageviewsByDayOfWeek order by prependNumber(Day_of_week)")

Wikipedia seems to get significantly more traffic on Mondays than other days of the week. Hmm...

### Can you visualize both the mobile and desktop site requests in a line chart to compare traffic between both sites by day of the week?

First, sum by DOW for each site (mobile/desktop)

In [ ]:
val viewsByDayOfWeekDF = pageviewsTimestampDF
  .groupBy(date_format(($"timestamp"), "E").alias("Day of week"), $"site")
  .sum().withColumnRenamed("sum(requests)", "total requests")
  .select(prependNumberUDF($"Day of week"), $"site", $"total requests")
  .orderBy("UDF(Day of week)")
  .toDF("DOW", "site", "total_requests")

viewsByDayOfWeekDF

Now, graph the mobile site requests:

In [ ]:
val mobileViewsByDayOfWeekDF = viewsByDayOfWeekDF.filter("site = 'mobile'")
  .select($"DOW", $"total_requests")
  .orderBy($"DOW")
  .toDF("DOW", "mobile_requests")

// Cache this DataFrame
mobileViewsByDayOfWeekDF.cache()

mobileViewsByDayOfWeekDF

For the visualization part we will use Graphical Widgets

In [ ]:
BarChart(mobileViewsByDayOfWeekDF, fields=Some(("DOW", "mobile_requests")))

Now, we are ready to plot our mobile requests
Next, graph the desktop site requests

In [ ]:
val desktopViewsByDayOfWeekDF = viewsByDayOfWeekDF.filter("site = 'desktop'")
.select($"DOW", $"total_requests")
  .orderBy($"DOW")
  .toDF("DOW", "desktop_requests")

// Cache this DataFrame
desktopViewsByDayOfWeekDF.cache()

desktopViewsByDayOfWeekDF

In [ ]:
BarChart(desktopViewsByDayOfWeekDF, fields=Some(("DOW", "desktop_requests")))

Now that we have two DataFrames (one for mobile views by day of week and another for desktop views), let's join both of them to create one line chart to visualize mobile vs. desktop page views:

In [ ]:
val allTogetherNowDF = mobileViewsByDayOfWeekDF.join(desktopViewsByDayOfWeekDF, "DOW")
allTogetherNowDF

In [ ]:
BarChart(viewsByDayOfWeekDF, fields=Some(("DOW", "total_requests")), groupField=Some("site"))

### Why is there so much more traffic on Monday vs. other days of the week?

***Challenge:*** Can you figure out exactly why there was so much more traffic on Mondays?

In [ ]:
// Type in your answer here...

In [ ]:
// first, check how many request per each day of year
// we see that one day have much more requests then the rest
// why does it have so many requests?
// what day of the year is the 110th day?
// its a monday :) so we are getting close
// it seems that this spesific day has duplicated data
// so monday didnt really have a spike!

// btw, the dip on sunday is becouse we have less sundays in the dataset :)